In [1]:
from IngestDrugSynonyms import IngestDrugSynonyms

In [2]:
import pandas as pd

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

/home/ben/Projects/Graphs4GoodHackathon/ProjectDomino/.Domino/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
drugSynonym = IngestDrugSynonyms()

In [8]:
drugSynonym.auto_get_and_clean_data()

2020-04-09 17:57:44,679 - > 403 studies found by 'covid-19' keyword
2020-04-09 17:58:07,131 - > 403 studies found by 'SARS-CoV-2' keyword
2020-04-09 17:58:29,969 - > 332 studies found by 'coronavirus' keyword
2020-04-09 17:58:46,243 - > 1050 distinct studies found


In [11]:
drug_vocab = drugSynonym.drug_vocab
drugs_and_syms:list = list(drug_vocab.keys())

drugs_and_syms.extend( item for key in drug_vocab.keys() if isinstance(drug_vocab[key],list) for item in drug_vocab[key])

In [12]:
len(drugs_and_syms)

39481

In [ ]:
import time

In [20]:
studies = drugSynonym.all_studies_df
appeared_in_edges:list = []
start = time.time()
print(start)
count = 0
prev = 0
for drug in drugs_and_syms:
    
    for index,row in studies.iterrows():
        if drug in row["intervention"]:
            appeared_in_edges.append((drug,row["trial_id"]))
    count += 1
    if count > prev + 1000:
        print(time.time())
        print(time.time() - start)
        prev = count
            
end = time.time()
print(end - start)
print(end)


1586419734.402499
1586419818.246048
83.84361219406128
1586419901.5618382
167.15940022468567
1586419985.5537164
251.15127801895142
1586420069.5865138
335.1840934753418
1586420153.0173287
418.61490273475647
1586420235.5331979
501.13076305389404
1586420318.3730078
583.9705700874329
1586420401.3945472
666.9921083450317
1586420484.51531
750.1128664016724
1586420568.2253563
833.8229167461395
1586420651.7354705
917.3330311775208
1586420735.1248071
1000.7223672866821
1586420818.3368707
1083.9344317913055
1586420901.8185306
1167.4160959720612
1586420986.0851562
1251.6827161312103
1586421070.07279
1335.6703515052795
1586421153.7543426
1419.351904630661
1586421237.545393
1503.1429777145386
1586421320.8366966
1586.4342572689056
1586421404.006324
1669.6038839817047
1586421487.4593859
1753.0569460391998
1586421570.824374
1836.4219362735748
1586421654.610131
1920.20769405365
1586421737.9467065
2003.5442702770233
1586421821.3660948
2086.9636595249176
1586421906.288895
2171.886456489563
1586421991.4019

In [21]:
start = time.time()
appeared_in_edges:list = [(drug,row["trial_id"]) for drug in drugs_and_syms for index,row in studies.iterrows() if drug in row["intervention"]]
end = time.time()
print(end - start)

3269.469130039215


In [22]:
39481 * 83

3276923

In [ ]:
GPU?